# Classiq code for discrete quantum walk

This notebook shows how to generate data for discrete quantum walk using `classiq`.

In [1]:
import time

from classiq import *

SIZE = 6
MAX_WIDTH = 40
constraints = Constraints(optimization_parameter="cx", max_width=MAX_WIDTH)

In [2]:
# mcx from control on X


@qfunc
def my_mcx(x: QNum, y: QBit):
    control(x == 2**x.size - 1, lambda: X(y))


# define increment circuit as an MCX cascade
@qfunc
def increment(x: QArray[QBit]):
    repeat(x.len - 1, lambda i: my_mcx(x[0 : x.len - 1 - i], x[x.len - 1 - i]))
    X(x[0])


@qfunc
def single_step_walk(
    x: QNum,  # position
):
    coin = QNum("coin")
    allocate(1, coin)
    H(coin)
    control(coin == 0, lambda: increment(x)),
    control(coin == 1, lambda: invert(lambda: increment(x)))

In [3]:
from classiq import CustomHardwareSettings, Preferences

preferences = Preferences(
    custom_hardware_settings=CustomHardwareSettings(basis_gates=["cx", "u"]),
    transpilation_option="custom",
    debug_mode=False,
)

## Example for getting a data point

In [4]:
start_time = time.time()


@qfunc
def main(x: Output[QNum]):
    allocate_num(SIZE, False, 0, x)
    single_step_walk(x)


qmod = create_model(main, constraints=constraints, preferences=preferences)
qprog = synthesize(qmod)
compilation_time = time.time() - start_time

quantum_program = QuantumProgram.from_qprog(qprog)
width = quantum_program.data.width
depth = quantum_program.transpiled_circuit.depth
cx_counts = quantum_program.transpiled_circuit.count_ops["cx"]

print(f"==== classiq for {SIZE}==== time {compilation_time}")

==== classiq for 6==== time 4.795164346694946
